In [1]:
## Test quadprog
#import daqp
import numpy as np
import pandas as pd
from math import comb
from ctypes import * 
import ctypes.util
import plotly.graph_objs as go
from collections import defaultdict
from plotly.offline import iplot
import itertools
import gudhi
import matplotlib.pyplot as plot
from numpy import genfromtxt
import math
def create_vector(a,b,step):
    new_vec = []
    curr_val = a
    while curr_val < b:
        new_vec.append(curr_val)
        curr_val += step
        
    return new_vec
from DensiTDA import alphatools 

In [2]:
Theta = create_vector(-3.1415/2,3.1415/2,0.2)
Phi = create_vector(0,2*3.1415,0.2)

Theta_, Phi_ = np.meshgrid(Theta, Phi)

Theta_ = Theta_.flatten()
Phi_ = Phi_.flatten()

In [3]:
def parametrized_function_2(u,v):
    
    return math.cos(u)*math.sin(v), math.sin(u)*math.sin(v), math.cos(v)

zfunc = np.vectorize(parametrized_function_2)

X_, Y_, Z_ = zfunc(Theta_, Phi_)

In [4]:
S = np.vstack((X_, Y_, Z_)).T

n = len(S) 

P = np.zeros((n,1))
alpha = 0.3
D = 3

In [5]:
DF = pd.DataFrame(S) 
DF.to_csv("sphere.csv")

In [6]:
# P = [4,4,4,4,1]
# S = np.array([[1,-1,-1],[-1,1,-1],[-1,-1,1],[1,1,1],[2,2,2]])
# alpha = 100
# D = 1

In [9]:
my_alpha_complex = alphatools.compute_alpha_complex(S, P, alpha, D)

Generating 1-Dimensional Weighted Cech Complex


100%|███████████████████████████████| 262144/262144 [00:00<00:00, 311880.50it/s]


	Total Edges of Cech Graph:  41970
	Highest Degree of Cech Graph:  195
Begin Computing Alpha Complex
Preprocessing Dual Matrices:  512


100%|████████████████████████████████████████| 512/512 [00:01<00:00, 366.62it/s]


*********** BEGIN DIMENSION 0 ***********
	Possible Facets:  512


...

	Final Number of Facets:  2442
*********** BEGIN DIMENSION 2 ***********
Estimating Number of Facets for dimension  2
	Possible Facets:  10452


100%|██████████████████████████████████| 10452/10452 [00:00<00:00, 19311.54it/s]


	Final Number of Facets:  5898
*********** BEGIN DIMENSION 3 ***********
Estimating Number of Facets for dimension  3
	Possible Facets:  1178905


100%|██████████████████████████████| 1178905/1178905 [01:02<00:00, 18941.94it/s]


	Final Number of Facets:  16780


In [ ]:
filtration = []
for d in range(D + 1):
    curr_dimension_simplicies_and_vals = my_alpha_complex[d]
    for simplex, val_matrix in curr_dimension_simplicies_and_vals:
        filtration.append((simplex, val_matrix[0]))

In [ ]:
alphatools.draw_alpha_complex(my_alpha_complex, S, alpha, , points_only = False)

In [ ]:
example2 = gudhi.SimplexTree()
for d in range(D + 1):
    for facet, val in my_alpha_complex[d]:
        example2.insert(facet, filtration=val)
        
my_persistence_alpha = example2.persistence(homology_coeff_field = 2)

In [ ]:
gudhi.plot_persistence_barcode(my_persistence_alpha, legend=True)
plot.show()

gudhi.plot_persistence_diagram(my_persistence_alpha, legend=True)
plot.show()

gudhi.plot_persistence_density(my_persistence_alpha, legend=True)
plot.show()

In [ ]:
# for k, matrix in enumerate(bigA): 
#     print(k, matrix @ matrix.T) 
#     if k == 10:
#         break

In [ ]:
# k = 0
# for V, A in zip(bigV, bigA):
#     print(k, A @ S[k] - V.T)
#     if k == 10:
#         break
#     k += 1

In [ ]:
VRips = gudhi.RipsComplex(points = S, max_edge_length = alpha)
Rips_simplex_tree_sample = VRips.create_simplex_tree(max_dimension = 3) 
rips_generator = Rips_simplex_tree_sample.get_filtration()
my_persistence = list(rips_generator)

In [ ]:
# gudhi.plot_persistence_diagram(my_persistence, legend=True)
# plot.show()

In [ ]:
list(rips_generator)

In [ ]:
for simplex_1, val_1 in my_persistence: 
    toggle_found = False
    for simplex_2, val_2 in filtration: 
        if simplex_1 == simplex_2: 
            toggle_found = True 
            if abs(val_1 - val_2) > 0.001:
                print("mismatch", simplex_1, val_1, val_2) 
    if toggle_found == False: 
        print("Simplex not found: ", simplex_1)

In [ ]:
np.linalg.norm(S[12] - S[13])